# Ag-Analytics® - Elevation Index API 

Local elevation on a field can have a strong correlation with harvest yield. The correlation stems from the drainage dynamics that are present on a given field, with low areas being productive in dry years and high areas being productive in wet years. 

The Elevation Index API supports calculation of a set of elevation indices including field relative elevation, maximum slope, ,local Topographic Position Index(TPI), and local Topographic Roughness Index(TRI) for a given field elevation. 

### Required libraries

In [1]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/elevation-index

1). __raster__(string): Local path to the elevation raster the index will be calculated from
       
2). __index_list__(string): The elevation index of interest to retrieve.
    
    i. Relative_Elevation: Derived by calculating the z-score of every point in the field
    
    ii. Slop: The maximum slope of each point on the field is calculated by taking the gradient of the elevation for each point.
    
    iii.TPI: The topographic position index is a method of measuring the elevation of a given point with respect to a surrounding neighborhood of surrounding points. This index indicates the local extrema,or points of maximum and minimum elevation on a sub-field level. 
    
    iV.TRI:  The topographic Roughness index express the amount of elevation difference between adjacent points on local field

3). __Outside_Radius(optional)__: Outer radius of search area of each point. Used in TPI only.

        (Ex:Outside_Radius =11 the outer search radius is 11 * cellsize. When cellsize is 0.001 of the elevation raster, the radius is about 11 * 8 = 88 meter

4). __Inside_Radius(optional)__: Inner radius of search area. Used in TPI only.

5). __Search_Radius(optional)__: Radius of computational box. search_radius = 1 => 3x3 box, 2 => 5x5 box, etc. Used in TRI only



### Request in JSON

In [ ]:
#Path to raster in your local machine
rasterpath = r"C:\Users\test_raster.tif"

files = [('elevation_raster',(open(rasterpath, 'rb')))]
print(files)

#Parameters to create index
values = {'index_list':"['Slope']",
            'Outside_Radius' :'5',
            'Inside_Radius':'3',
            'Search_Radius' :'7',
            'Legend_Range': '10'
}

# Basic Header Pattern.
headers={'Content-Type':'application/json'}

# Header for using a subscription key.
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

### API Function

In [18]:
def ElevationIndex(files,values):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
        
        response = requests.post(url, files = files, data = values).json()
        #resp = json.loads(response)
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [19]:
IndexResponse = ElevationIndex(files,values)

{'Slope': 'raster_Slope_60326.tif', 'features': [[{'attributes': {'CellSize': [9.259259269220297e-05, -9.259259269220297e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-76.4984894, 42.44091207776192, -76.47552643701233, 42.455634299999986', 'Legend': [{'Area': '33.33 %', 'Count': 12550, 'CountAllPixels': 37649, 'Max': 0.0182556863874197, 'Mean': 0.009127915863693659, 'Min': 1.4533996761656454e-07, 'color': '#4d76d5'}, {'Area': '33.33 %', 'Count': 12549, 'CountAllPixels': 37649, 'Max': 0.0475408136844635, 'Mean': 0.0328982500359416, 'Min': 0.0182556863874197, 'color': '#e5cc78'}, {'Area': '33.33 %', 'Count': 12550, 'CountAllPixels': 37649, 'Max': 0.8181570172309875, 'Mean': 0.4328489154577255, 'Min': 0.0475408136844635, 'color': '#e9e9e9'}], 'Matrix': [159, 248], 'Max': 0.8181570172309875, 

### Output Index Summary

In [20]:
df=defaultdict(list)
for value in IndexResponse['features']:
    attributes = value[0]['attributes']
    df['Index'].append(attributes['Product'])
    df['Max'].append(float(attributes['Max']))
    df['Mean'].append(float(attributes['Mean']))
    df['Min'].append(float(attributes['Min']))
    df['pngb64'].append(attributes['pngb64'])

indexdf=pd.DataFrame.from_dict(df)
#indexdf = indexdf.sort_values(by = ['Area'], ascending=False).style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])
indexdf

,Index,Max,Mean,Min,pngb64
0,Slope,0.818157,0.057451,1.453400e-07,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."


### Displaying Index Image

In [21]:
#####################################
# Select one index to display. Must match one of the following: 
# ['Slope', 'Relative_Elevation', 'TPI', 'TRI']
Index = 'Slope'
######################################

In [22]:
#Select one index
Index = Index

#Image Legend
df=defaultdict(list)
for product in IndexResponse['features']:
    if Index == product[0]['attributes']['Product']:
        Legend = product[0]['attributes']['Legend']
for val in Legend:
    df['Max'].append(float(val['Max']))
    df['Mean'].append(float(val['Mean']))
    df['Min'].append(float(val['Min']))
    df['Color'].append(val['color'])
    df['Range'] = ['Low','Medium','High']
    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])

,Max,Mean,Min,Color,Range
0,0.0182557,0.00912792,1.4534e-07,#4d76d5,Low
1,0.0475408,0.0328983,0.0182557,#e5cc78,Medium
2,0.818157,0.432849,0.0475408,#e9e9e9,High


In [23]:
IndexImage = indexdf.loc[indexdf['Index'] == Index,'pngb64'].iloc[0]
Image(url = IndexImage, width= 500, height =500)

# GET Request

In [38]:
# Specify local path where file will be downloaded.
local_path = r"C:\<Path to download directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"

# Create GET request payload.
Index = 'Relative_Elevation'
values = {'filenames': [IndexResponse[Index],]}
print(values)

{'filenames': ['raster_RelativeElevation_84049.tif']}


In [39]:
def Elevation_Index_get(values, local_path):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
    
        download_path = os.path.join(local_path, values['filenames'][0])
        print(values)
     
        response = requests.get(url, params=values)
        open(download_path, 'wb').write(response.content)
        
        print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [ ]:
Elevation_Index_get(values, local_path)